# Blank-node network to RDF-star

This notebook converts the network modelled in `../data/knowledge-graph/vasari-kg.ttl` from a blank-node based version, to a network expressed in RDF-star notation.

It uses the [pyoxigraph library](https://pyoxigraph.readthedocs.io/en/stable/) in Python, if you do not have it installed yet, install it first using the next cell.

In [ ]:
!pip install pyoxigraph

In [3]:
import pyoxigraph as px
from rich import print

In [4]:
store = px.Store()
store.bulk_load('../data/knowledge-graph/vasari-kg.ttl', 'text/turtle')
print(len(store))

5837

In [5]:
edges = {}
for s,p,o,_ in store.quads_for_pattern(None, px.NamedNode('https://ISE-FIZKarlsruhe.github.io/vasari_network/property/isEdgeOf'), None):
    edges.setdefault(s, []).append(o)



In [6]:
x = [(p,o) for s,p,o,_ in store.quads_for_pattern(list(edges.keys())[0], None, None)]
print(x)
print(list(edges.values())[0])

[
    (
        <NamedNode value=https://ISE-FIZKarlsruhe.github.io/vasari_network/property/isEdgeOf>,
        <NamedNode value=https://ISE-FIZKarlsruhe.github.io/vasari_network/entity/Apelles>
    ),
    (
        <NamedNode value=https://ISE-FIZKarlsruhe.github.io/vasari_network/property/isEdgeOf>,
        <NamedNode 
value=https://ISE-FIZKarlsruhe.github.io/vasari_network/entity/Botticelli%2C%20Sandro%20%28Sandro%20di%20Botticello
%2C%20or%20Alessandro%20Filipepi%29>
    ),
    (
        <NamedNode value=https://ISE-FIZKarlsruhe.github.io/vasari_network/property/weight>,
        <Literal value=7.7567945 datatype=<NamedNode value=http://www.w3.org/2001/XMLSchema#float>>
    )
]

[
    <NamedNode value=https://ISE-FIZKarlsruhe.github.io/vasari_network/entity/Apelles>,
    <NamedNode 
value=https://ISE-FIZKarlsruhe.github.io/vasari_network/entity/Botticelli%2C%20Sandro%20%28Sandro%20di%20Botticello
%2C%20or%20Alessandro%20Filipepi%29>
]

In [7]:
out = px.Store()
for s,p,o,_ in store.quads_for_pattern(None, px.NamedNode('https://ISE-FIZKarlsruhe.github.io/vasari_network/property/weight'), None):
    edge = edges.get(s)
    a, b = edge
    t = px.Quad(a, px.NamedNode('https://ISE-FIZKarlsruhe.github.io/vasari_network/link'), b)
    out.add(t)
    q = px.Quad(t.triple, px.NamedNode('https://ISE-FIZKarlsruhe.github.io/vasari_network/property/weight'), o)
    out.add(q)
out.dump('vasari-network-rdf-star.nt', 'application/n-triples')

In [8]:
circled = px.Store()
circled.bulk_load('vasari-network-rdf-star.nt', 'application/n-triples')
for s,p,o,_ in circled.quads_for_pattern(None, None, None):
    if p == px.NamedNode('https://ISE-FIZKarlsruhe.github.io/vasari_network/link'):
        continue
    print(s,p,o)
    print(type(s))
    break

<https://ISE-FIZKarlsruhe.github.io/vasari_network/entity/Apelles> 
<https://ISE-FIZKarlsruhe.github.io/vasari_network/link> 
<https://ISE-FIZKarlsruhe.github.io/vasari_network/entity/Botticelli%2C%20Sandro%20%28Sandro%20di%20Botticello%2C%2
0or%20Alessandro%20Filipepi%29> <https://ISE-FIZKarlsruhe.github.io/vasari_network/property/weight> 
"7.7567945"^^<http://www.w3.org/2001/XMLSchema#float>

<class 'pyoxigraph.Triple'>